# 🔐 Secure Blob Access via API Management

This sample demonstrates implementing the **valet key pattern** with Azure API Management (APIM) to provide secure, time-limited access to blob storage without exposing storage account keys.

## ✨ What This Demo Does

1. **Creates storage infrastructure** with a storage account and container
2. **Configures APIM** with managed identity and Storage Blob Data Reader permissions  
3. **Deploys sample files** using Infrastructure as Code (IaC)
4. **Implements secure API** that generates time-limited blob access URLs
5. **Tests the solution** with authorized and unauthorized access scenarios

## 🧩 Lab Components

This lab sets up:
- A simple Azure Storage account with LRS redundancy
- A blob container with a sample text file
- APIM managed identity with Storage Blob Data Reader permissions
- An API that generates secure blob access URLs using the valet key pattern
- Sample files: a text file for testing

## ⚙️ Configuration

1. Decide which of the [Infrastructure Architectures](../../README.md#infrastructure-architectures) you wish to use.
    1. If the infrastructure _does not_ yet exist, navigate to the desired [infrastructure](../../infrastructure/) folder and follow its README.md.
    1. If the infrastructure _does_ exist, adjust the `user-defined parameters` in the _Initialize notebook variables_ below. Please ensure that all parameters match your infrastructure.

### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment.

**Modify entries under _1) User-defined parameters_**. The APIs are pre-configured for the valet key pattern implementation.

In [ ]:
import utils
import time
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 2
deployment  = INFRASTRUCTURE.SIMPLE_APIM  # This sample works with all infrastructures
tags        = ['secure-blob-access', 'valet-key', 'storage', 'jwt', 'authz']
api_prefix  = 'blob-'  # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)
supported_infrastructures = [INFRASTRUCTURE.SIMPLE_APIM, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.AFD_APIM_PE]
utils.validate_infrastructure(deployment, supported_infrastructures)

# Set up the signing key for the JWT policy
jwt_key_name = f'JwtSigningKey{int(time.time())}'
jwt_key_value, jwt_key_value_bytes_b64 = utils.generate_signing_key()
utils.print_val('JWT key value', jwt_key_value)
utils.print_val('JWT key value (base64)', jwt_key_value_bytes_b64)

# 3) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(jwt_key_name, jwt_key_value_bytes_b64, True),
    NamedValue('HRMemberRoleId', HR_MEMBER_ROLE_ID)
]

# 4) Set up the policy fragments
pf_authx_hr_member_xml = utils.read_policy_xml('./pf-authx-hr-member.xml').format(
    jwt_signing_key = '{{' + jwt_key_name + '}}',
    hr_member_role_id = '{{HRMemberRoleId}}'
)

# Reuse shared policy fragments
pf_authz_match_any_xml = utils.read_policy_xml('../../shared/apim-policies/fragments/pf-authz-match-any.xml')

pfs: List[PolicyFragment] = [
    PolicyFragment('AuthX-HR-Member', pf_authx_hr_member_xml, 'Authenticates and authorizes users with HR Member role.'),
    PolicyFragment('AuthZ-Match-Any', pf_authz_match_any_xml, 'Authorizes if any of the specified roles match the JWT role claims.')
]

# 5) Define constants
container_name = 'samples'

blob_get_policy_xml = utils.read_and_modify_policy_xml('./blob-get-operation.xml', {
    'hr_member_role_id': '{{HRMemberRoleId}}',
    'storage_account_name': '{{storage-account-name}}',
    'container_name': container_name
})

# 6) Define the APIs and their operations and policies

# Define template parameters for the blob name
blob_template_parameters = [
    {
        "name": "blob-name",
        "description": "The name of the blob to access",
        "type": "string",
        "required": True
    }
]

# Secure Blob Access API
blob_get = GET_APIOperation2('GET', 'GET', '/{blob-name}', 'Gets the blob access valey key', blob_get_policy_xml,
    templateParameters = blob_template_parameters)
                             
secure_blob_api = API(name = 'secure-blob-access', displayName = 'Secure Blob Access API', path = f'/{api_prefix}secure-files', 
    description = 'API for secure access to blob storage using the valet key pattern', operations = [blob_get], tags = tags)

# APIs Array
apis: List[API] = [secure_blob_api]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the Bicep deployment into the previously-specified resource group. This will:
- Create a storage account with LRS redundancy
- Set up a blob container with private access
- Grant APIM's managed identity Storage Blob Data Reader permissions
- Deploy the secure blob access API

A Bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]},
    'policyFragments': {'value': [pf.to_dict() for pf in pfs]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    storage_account_name = output.get('storageAccountName', 'Storage Account Name')
    storage_endpoint = output.get('storageAccountEndpoint', 'Storage Endpoint')
    container_name = output.get('blobContainerName', 'samples')

utils.print_ok('Deployment completed')

### 📁 3. Sample Files Created by Infrastructure

The infrastructure deployment automatically creates sample files in the blob storage container using a deployment script. No manual upload is required!

**Files created during deployment:**
- `sample.txt` - A text file demonstrating secure access to text content

The deployment script uses a user-assigned managed identity with **Storage Blob Data Contributor** permissions to upload these files as part of the Infrastructure as Code (IaC) process. This approach is more secure and reliable than manual uploads.

✅ **Benefits of IaC-based file creation:**
- Consistent deployment across environments
- No manual intervention required
- Secure managed identity authentication
- Files are available immediately after infrastructure deployment

### ✅ 4. Test the Secure Blob Access with Authentication

Test the secure blob access API to verify both the authentication/authorization and valet key pattern implementation. We'll:
1. Create JWT tokens for authorized and unauthorized users
2. Test API access with valid authentication
3. Test access denial for unauthorized users  
4. Verify direct blob access using the valet key pattern

The sample file (`sample.txt`) was automatically created during the infrastructure deployment using a Bicep deployment script.

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URL will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimjwt import JwtPayload, SymmetricJwtToken
import requests
import json

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door
endpoint_url = apim_gateway_url
utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

if afd_endpoint_url:
    endpoint_url = afd_endpoint_url
    utils.print_message(f'Using Azure Front Door URL: {afd_endpoint_url}', blank_above = True)
else:
    utils.print_message(f'Using APIM Gateway URL: {apim_gateway_url}', blank_above = True)

# 1) Test with authorized user (has blob access role)
utils.print_message('Testing with Authorized User', blank_above = True)

# Create a JWT with the HR Member role (blob access)
jwt_payload_authorized = JwtPayload(
    subject = 'user123', 
    name = 'Alice HR Member',
    roles = [HR_MEMBER_ROLE_ID]
)
encoded_jwt_token_authorized = SymmetricJwtToken(jwt_key_value, jwt_payload_authorized).encode()
print(f'JWT token (Authorized): {encoded_jwt_token_authorized}')

# Set up APIM requests with authorized JWT
reqsApimAuthorized = ApimRequests(endpoint_url)
reqsApimAuthorized.headers['Authorization'] = f'Bearer {encoded_jwt_token_authorized}'

# Test sample file access
print("\n🔒 Getting secure access for sample.txt with authorized user...")
response = reqsApimAuthorized.singleGet('/secure-files/sample.txt', 
                             msg = 'Requesting secure access for sample.txt (authorized)')

# Handle response whether it's a response object or string
if response:
    if hasattr(response, 'status_code') and response.status_code == 200:
        try:
            access_info = response.json()
            print("\n📋 Secure Access Information:")
            print(f"   Authorized User: {access_info.get('authorized_user', 'N/A')}")
            print(f"   Secure URL: {access_info.get('secure_url', 'N/A')}")
            print(f"   Expires At: {access_info.get('expires_at', 'N/A')}")
            print(f"   Method: {access_info.get('access_method', 'N/A')}")
            print(f"   Note: {access_info.get('note', 'N/A')}")
            
            # Test direct access to the blob using the provided credentials
            if 'secure_url' in access_info and 'authorization_header' in access_info:
                print("\n🧪 Testing direct blob access...")
                headers = {'Authorization': access_info['authorization_header']}
                
                try:
                    blob_response = requests.get(access_info['secure_url'], headers = headers)
                    if blob_response.status_code == 200:
                        print("✅ Direct blob access successful!")
                        content_preview = blob_response.text[:200] + "..." if len(blob_response.text) > 200 else blob_response.text
                        print(f"📄 Content preview:\n{content_preview}")
                    else:
                        print(f"❌ Direct blob access failed: {blob_response.status_code}")
                except Exception as e:
                    print(f"❌ Error accessing blob directly: {str(e)}")
        except (json.JSONDecodeError, AttributeError):
            print("❌ Failed to parse JSON response or response is not in expected format")
    else:
        print(f"ℹ️ Response received: {response}")

# 2) Test with unauthorized user (no blob access role)
utils.print_message('Testing with Unauthorized User', blank_above = True)

# Create a JWT without blob access role
jwt_payload_unauthorized = JwtPayload(
    subject = 'user789', 
    name = 'Bob Regular User',
    roles = ['some-other-role-id']
)
encoded_jwt_token_unauthorized = SymmetricJwtToken(jwt_key_value, jwt_payload_unauthorized).encode()
print(f'JWT token (Unauthorized): {encoded_jwt_token_unauthorized}')

# Set up APIM requests with unauthorized JWT
reqsApimUnauthorized = ApimRequests(endpoint_url)
reqsApimUnauthorized.headers['Authorization'] = f'Bearer {encoded_jwt_token_unauthorized}'

# Test access denial
print("\n🚫 Testing access denial...")
reqsApimUnauthorized.singleGet('/secure-files/sample.txt', 
                               msg = 'Requesting secure access for sample.txt (unauthorized - expect 403)')

# 3) Test with no authentication
utils.print_message('Testing with No Authentication', blank_above = True)
reqsApimNoAuth = ApimRequests(endpoint_url)
reqsApimNoAuth.singleGet('/secure-files/sample.txt', 
                         msg = 'Requesting secure access for sample.txt (no auth - expect 401)')

# Summary
utils.print_message('Secure Blob Access Pattern Summary', blank_above = True)
print(f"✅ Infrastructure deployed successfully")
print(f"✅ Sample files created automatically via Bicep deployment script")
print(f"✅ API Management configured with managed identity for blob access")
print(f"✅ Valet key pattern implemented for secure file access")
print(f"   Authorized users with role '{HR_MEMBER_ROLE_ID}' (HR Member) can request secure URLs")
print(f"   Unauthorized users receive appropriate error responses")
print(f"   Direct blob access works with time-limited, secure credentials")
print(f"\n🎯 The valet key pattern allows secure, scalable file access without exposing storage keys!")

# Display information about the sample files created during deployment
print(f"\n📂 Sample files created during deployment:")
test_files = ['sample.txt']
for filename in test_files:
    print(f"   📄 {filename}")

utils.print_ok("Sample deployment and API configuration completed!")
print()
print("🚀 Next Steps:")
print("   1. Test the API with proper JWT tokens")
print("   2. Verify file access through generated URLs") 
print("   3. Monitor API usage and performance")
print("   4. Customize policies based on your requirements")